# Introdução a Análise Exploratória de Dados (EDA)
* Autor: João Tedeschi 

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ITalents/CD-Introducao-a-Ciencia-de-dados)

Caso precise instalar alguma biblioteca, utilize o comando `!pip install pandas numpy matplotlib seaborn duckdb plotly` numa célula de python no Jupyter Notebook.

A __Análise Exploratória de Dados (EDA)__ é uma abordagem para analisar dados para resumir suas principais características, geralmente com métodos de estatística descritiva e métodos visuais. Um dos principais objetivos da EDA é obter uma melhor compreensão dos dados e do que eles podem nos dizer sobre os fenômenos que eles representam.

Além disso, a EDA é uma boa maneira de detectar erros e anomalias nos dados, como valores ausentes ou discrepantes, e também é uma boa maneira de obter uma ideia de quais modelos podem ser adequados para os dados.

A EDA é uma parte importante do processo de modelagem de dados, pois ajuda a garantir que os dados sejam adequados para o modelo que você deseja ajustar. Se você não fizer uma EDA, poderá acabar com um modelo que não se ajusta bem aos dados ou, pior, com um modelo que se ajusta bem aos dados que você tem, mas não aos dados que você deseja prever.

Dito isso vamos começar a nossa EDA. Primeiro vamos importar as bibliotecas que vamos utilizar.

In [ ]:
!git clone https://github.com/ITalents/CD-Introducao-a-Ciencia-de-dados.git

## Importando as bibliotecas

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob

Configurando a visualização do `pandas` para mostrar todas as colunas

```python
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
    
```



In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

%matplotlib inline


## Bibliotecas

<!-- tabela com nome e uso de bibliotecas -->

Em resumo, as bibliotecas que vamos utilizar são:

| Biblioteca | Uso |
| --- | --- |
| [pandas](https://pandas.pydata.org/docs/getting_started/index.html) | Manipulação e análise de dados |
| [numpy](https://numpy.org/doc/stable/) | Manipulação de arrays e matrizes multidimensionais |
| [matplotlib](https://matplotlib.org/) | Visualização de dados |
| [seaborn](https://seaborn.pydata.org/) | Visualização de dados |
| [glob](https://docs.python.org/3/library/glob.html) | Manipulação de arquivos e diretórios |
| [duckdb](https://duckdb.org/docs/api/python/dbapi) | Banco de dados relacional |

---
### Resumo

__Pandas__

Pandas é uma biblioteca de código aberto que fornece estruturas de dados e ferramentas de análise de dados de alto desempenho e fáceis de usar para a linguagem de programação Python.

__Numpy__

Numpy é uma biblioteca para a linguagem Python que suporta arrays e matrizes multidimensionais, possuindo uma larga coleção de funções matemáticas para trabalhar com estas estruturas. Além disso, numpy possui um conjunto amplo de funções matemáticas de alto nível que operam nestes arrays.

__Matplotlib__

Matplotlib é uma biblioteca de software para criação de gráficos e visualizações de dados em geral, feita para e da linguagem de programação Python e sua extensão de matemática NumPy.

__Seaborn__

Seaborn é uma biblioteca de visualização de dados Python baseada em matplotlib. Ele fornece uma interface de alto nível para desenhar gráficos estatísticos atraentes e informativos.

__Glob__

Glob é um módulo nativo do python que fornece uma função para criar listas de arquivos a partir de buscas em diretórios usando caracteres curinga. Ele é usado para encontrar arquivos e diretórios correspondentes a um determinado padrão, como quando você está trabalhando com vários arquivos de dados.

__Duckdb__

Duckdb é uma biblioteca de banco de dados relacional que pode ser utilizada para armazenar e manipular dados. Ela é uma alternativa ao SQLite, porém com algumas vantagens, como por exemplo, ser mais rápida e possuir suporte a tipos de dados mais complexos.



## Importando os dados

Para realizar a EDA vamos utilizar um conjunto de dados de um estudo de caso de uma empresa de telecomunicações. O conjunto de dados contém informações sobre clientes de uma empresa de telecomunicações e se eles deixaram a empresa (churn). O conjunto de dados também contém informações sobre os serviços que cada cliente assinou, como telefone, várias linhas, internet, segurança online, backup online, proteção de dispositivo, suporte técnico e streaming de TV e filmes. Por fim, há informações demográficas sobre os clientes, como sexo, faixa etária e se eles têm parceiros e dependentes. Fonte: IBM

Vamos importar os dados utilizando a biblioteca pandas.

__Referência para Download:__ [Kaggle](https://www.kaggle.com/datasets/ylchang/telco-customer-churn-1113)

In [ ]:
# Importando a biblioteca DuckDB
import duckdb

# Conectando-se ao banco de dados em memória do DuckDB
duckdb_conn = duckdb.connect(database=':memory:', read_only=False)

# Looping através de todos os arquivos .xlsx na pasta 'dados'
for arquivo in glob.glob('/content/CD-Introducao-a-Ciencia-de-dados/EDA/dados/*.xlsx'):
    # Imprimindo o nome do arquivo atual
    print(arquivo)
    
    # Extraindo o nome do arquivo sem a extensão e usando como nome da tabela no banco de dados
    nome_arquivo = arquivo.split('/')[1].split('.')[0].split('_')[-1]
    
    # Lendo o arquivo Excel com o pandas e registrando-o como uma tabela no DuckDB
    dataframe = pd.read_excel(arquivo)
    duckdb_conn.register('tb_'+nome_arquivo, dataframe)



## Inspecionando dados

Agora que subimos os dados para o banco de dados, vamos inspecionar os dados para ver suas características. Para isso, vamos realizar algumas consultas (ou *queries*) em `SQL`.

<!-- explicando um SELECT básico -->

```sql
SELECT * FROM tb_services;
```

Essa query está selecionando todas (denotado pelo *wildcard* `*` ) as colunas e linhas da tabela `tb_services`.

Usando a conexão com o banco DuckDB instanciada como `duckdb_conn` e o método `execute()` podemos executar a query e armazenar o resultado em um dataframe do pandas.


o Método `info()` do pandas mostra informações sobre o dataframe, como o nome das colunas, o tipo de dado de cada coluna e a quantidade de valores não nulos.


In [ ]:
# Criando uma query para extrair os dados de todas as tabelas
# Excluindo a coluna 'Count' das tabelas

query = """
SELECT * EXCLUDE (Count) FROM tb_status
LEFT JOIN tb_services USING ("Customer ID")
LEFT JOIN tb_demographics USING ("Customer ID")
LEFT JOIN tb_location USING ("Customer ID")
LEFT JOIN tb_population USING ("Zip Code")
"""


In [ ]:
# Executando a query e armazenando o resultado em um DataFrame
dataframe_telco = duckdb_conn.execute(query).fetch_df()

# o Método .head() exibe as 5 primeiras linhas do DataFrame
dataframe_telco.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Count                              7043 non-null   int64  
 2   Quarter                            7043 non-null   object 
 3   Referred a Friend                  7043 non-null   object 
 4   Number of Referrals                7043 non-null   int64  
 5   Tenure in Months                   7043 non-null   int64  
 6   Offer                              3166 non-null   object 
 7   Phone Service                      7043 non-null   object 
 8   Avg Monthly Long Distance Charges  7043 non-null   float64
 9   Multiple Lines                     7043 non-null   object 
 10  Internet Service                   7043 non-null   object 
 11  Internet Type                      5517 non-null   objec

In [15]:
services = duckdb_conn.execute('''SELECT * FROM tb_services''').fetch_df()



services.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 30 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Customer ID                        7043 non-null   object 
 1   Count                              7043 non-null   int64  
 2   Quarter                            7043 non-null   object 
 3   Referred a Friend                  7043 non-null   object 
 4   Number of Referrals                7043 non-null   int64  
 5   Tenure in Months                   7043 non-null   int64  
 6   Offer                              3166 non-null   object 
 7   Phone Service                      7043 non-null   object 
 8   Avg Monthly Long Distance Charges  7043 non-null   float64
 9   Multiple Lines                     7043 non-null   object 
 10  Internet Service                   7043 non-null   object 
 11  Internet Type                      5517 non-null   objec

In [ ]:
# Quantida de Linhas e Colunas da Tabela
services.shape

### Tipos de dados

Existem vários tipos de dados que podem ser armazenados em um banco de dados. Os tipos de dados mais comuns são:
* Dados numéricos, como números inteiros (`int`) e números de ponto flutuante (`float`)
* Dados de texto, como strings (`str`)
* Dados de data e hora, como data, hora, data e hora e intervalo de tempo
* Dados booleanos, que podem ser verdadeiros ou falsos (`True` ou `False`)

Esses dados podem representar **variáveis** ou **atributos**. Uma variável é uma característica de um objeto, enquanto um atributo é uma característica de uma entidade. Por exemplo, se você tiver uma tabela de clientes, cada linha da tabela será um cliente e cada coluna será uma variável ou atributo do cliente. As variáveis podem ser de diferentes tipos de dados, como numéricas ou de texto.

#### Variáveis numéricas

Variáveis numéricas podem ser **contínuas** ou **discretas**. Variáveis contínuas podem assumir qualquer valor dentro de um intervalo, enquanto variáveis discretas podem assumir apenas valores inteiros. Por exemplo, a renda de uma pessoa é uma variável contínua, pois pode assumir qualquer valor dentro de um intervalo. Por outro lado, o número de filhos de uma pessoa é uma variável discreta, pois só pode assumir valores inteiros.	

#### Variáveis categóricas

Variáveis categóricas podem assumir valores **Nominais** ou **Ordinais**. Variáveis nominais não têm uma ordem específica, enquanto variáveis ordinais têm uma ordem específica. Por exemplo, o sexo de uma pessoa é uma variável nominal, pois não há uma ordem específica entre os valores. Por outro lado, o nível de educação de uma pessoa é uma variável ordinal, pois há uma ordem específica entre os valores (por exemplo, ensino fundamental, ensino médio, ensino superior).


#### Valores nulos

Valores nulos podem indicar que os dados estão faltando ou que não existem. Por exemplo, se um cliente não tiver um número de telefone, o valor do número de telefone será nulo. Os valores nulos podem ser representados por `NULL` ou `NaN` (Not a Number).

### Estatísticas descritivas

O método `describe()` do `pandas` mostra estatísticas descritivas sobre as **colunas numéricas** do dataframe, como a média, desvio padrão, valor mínimo e máximo, etc.

```python	
services.describe()
```

Observe que usando esse método, já conseguimos ter algum detalhamento adicional sobre a distribuição dos dados. Mais adianta vamos utilizar o `seaborn` para visualizar essas distribuições.



#### Medidas de Tendência Central
* Mínimo: é o menor valor em um conjunto de dados.
* Máximo: é o maior valor em um conjunto de dados.

* Média: é a soma de todos os valores dividido pelo número de valores.
<!-- Mathjax -->
$$\mu = \frac{\sum_{i=1}^{n} x_i}{n}$$

* Mediana: é o valor que separa a metade superior da metade inferior de uma distribuição de dados.
<!-- Mathjax -->

$$ \frac{1}{2} (x_{\frac{n+1}{2}} + x_{\frac{n+1}{2}+1})$$


* Moda: é o valor que aparece com mais frequência em um conjunto de dados.
<!-- Mathjax -->
$$ \text{Moda} = \text{valor mais frequente}$$

#### Medidas de Dispersão
* Variância: é a média dos quadrados das diferenças entre os valores e a média.
<!-- Mathjax -->
$$\sigma^2 = \frac{\sum_{i=1}^{n} (x_i - \mu)^2}{n}$$

* Desvio Padrão: é a raiz quadrada da variância.
<!-- Mathjax -->
$$\sigma = \sqrt{\frac{\sum_{i=1}^{n} (x_i - \mu)^2}{n}}$$

* Coeficiente de Variação: é o desvio padrão dividido pela média.
<!-- Mathjax -->

$$CV = \frac{\sigma}{\mu}$$




In [ ]:
services.describe()

### Descritivo de colunas categóricas

Para inspecionar as colunas categóricas, podemos usar o método `value_counts()` do `pandas`. Esse método mostra a quantidade de valores únicos em cada coluna categórica.

```python
services['internet_service'].value_counts()
```

Foi definida uma função para facilitar a inspeção de todas as colunas categóricas, chamada `descreve_colunas` que recebe como parâmetro um dataframe e retorna um dataframe com as colunas categóricas e a quantidade de valores únicos em cada uma delas.


In [ ]:
import re

def descreve_colunas(dataframe : pd.DataFrame):
    """ Função para descrever as colunas categóricas de um dataframe. 
        Ela imprime os valores únicos e a quantidade de valores únicos de cada coluna.
        Só aceita objetos do tipo pandas.DataFrame."""
    
    # Contagem de valores unicos para colunas categoricas
    cat_cols = [col for col in dataframe.columns if dataframe[col].dtype not in ['int64', 'float64']]

    # Looping para imprimir os valores unicos de cada coluna categorica
    print('-----------------------')
    print('Colunas categóricas:')
    print(cat_cols)
    print('-----------------------')

    for col in cat_cols:
        # contém o valor ID? Se sim, não imprime
        if not re.search('id', col, re.IGNORECASE):  
            # distribuição de frequencia
            print(f'Distribuição de frequência:\n{dataframe[col].value_counts()}')
            print(f'Valores únicos: {dataframe[col].unique()}')
            print(f'Quantidade de valores unicos: {len(dataframe[col].unique())}')
            print(f'Valores nulos: {dataframe[col].isnull().sum()}')
            print('-----------------------')

descreve_colunas(services)



## Análise univariada

A análise univariada é a análise de uma única variável (como o nome já diz). Ela pode ser feita para variáveis numéricas ou categóricas. Para variáveis numéricas, podemos visualizar a distribuição dos dados usando um histograma ou um boxplot. Para variáveis categóricas, podemos visualizar a distribuição dos dados usando um gráfico de barras.

### Variáveis numéricas

Visualizando a distribuição dos dados usando um histograma.

```python
services['Monthly Charge'].hist()
```

Visualizando a distribuição dos dados usando um boxplot.

```python
services['Monthly Charge'].plot(kind='box')
```

#### Histograma

Um histograma é um gráfico de barras que mostra a frequência de cada valor. O eixo x mostra os valores e o eixo y mostra a frequência. Por exemplo, se o eixo x mostra os valores de 0 a 10 e o eixo y mostra a frequência de cada valor, então o histograma mostra quantas vezes cada valor de 0 a 10 aparece nos dados.

#### Boxplot

Um boxplot é um grafico quem mostra a dispersão daquela variável. Ele mostra a mediana, o primeiro e terceiro quartil, os valores mínimo e máximo e os valores discrepantes (outliers). Os valores discrepantes são valores que estão muito distantes dos outros valores. Por exemplo, se o valor mínimo for 0 e o valor máximo for 100, então um valor discrepante seria 1000.

##### Quartis

Quartis são medidas descritivas que dividem os dados em quatro partes iguais. O primeiro quartil (Q1) é o valor que divide os dados em 25% e 75%. O segundo quartil (Q2) é o valor que divide os dados em 50% e 50%, **a mediana é o segundo quartil**. O terceiro quartil (Q3) é o valor que divide os dados em 75% e 25%. O primeiro quartil é a mediana dos dados menores que a mediana e o terceiro quartil é a mediana dos dados maiores que a mediana.

** Distância interquartil (IQR) **
A distância interquartil (IQR) é a diferença entre o terceiro e o primeiro quartil. Por exemplo, se o primeiro quartil for 10 e o terceiro quartil for 20, então o IQR será 10.

In [ ]:
# Histograma da coluna monthly_charges
# plt.figure é uma função do matplotlib para criar uma figura

plt.figure(figsize=(10, 6))
# plt.hist é uma função do matplotlib para criar um histograma com 50 bins
plt.hist(services['Monthly Charge'], bins=50)

# plt.title é uma função do matplotlib para adicionar um título ao gráfico
plt.title('Distribuição de frequência da coluna Monthly Charge')
# anotar a média, mediana e moda com plt.axvline e plt.legend
plt.axvline(services['Monthly Charge'].mean(), color='red', linestyle='dashed', linewidth=1)
plt.axvline(services['Monthly Charge'].median(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(services['Monthly Charge'].mode()[0], color='yellow', linestyle='dashed', linewidth=1)
plt.legend(['Média', 'Mediana', 'Moda'])

# plt.xlabel é uma função do matplotlib para adicionar um título ao eixo x
plt.xlabel('monthly_charges')
# plt.ylabel é uma função do matplotlib para adicionar um título ao eixo y
plt.ylabel('Frequência')
# plt.show é uma função do matplotlib para mostrar o gráfico
plt.show()

In [ ]:
# Boxplot da coluna Monthly Charge
plt.figure(figsize=(10, 6))
# plt.boxplot é uma função do matplotlib para criar um boxplot
plt.boxplot(services['Monthly Charge'])
# anotar a média, mediana e moda com plt.axvline e plt.legend
plt.axhline(services['Monthly Charge'].mean(), color='red', linestyle='dashed', linewidth=1)
plt.axhline(services['Monthly Charge'].median(), color='green', linestyle='dashed', linewidth=1)
plt.axhline(services['Monthly Charge'].mode()[0], color='yellow', linestyle='dashed', linewidth=1)
plt.legend(['Média', 'Mediana', 'Moda'])

# anotar os quartis com plt.text
plt.text(1.1, services['Monthly Charge'].quantile(0.25), 'Q1', fontsize=12)
plt.text(1.1, services['Monthly Charge'].quantile(0.5), 'Q2', fontsize=12)
plt.text(1.1, services['Monthly Charge'].quantile(0.75), 'Q3', fontsize=12)

plt.title('Boxplot da coluna Monthly Charge')
plt.show()

In [ ]:
# Criar um histograma e abaixo um boxplot horzontal para a coluna Total Charges subplot
plt.figure(figsize=(10, 6))
plt.subplot(2, 1, 1)
plt.hist(services['Total Charges'], bins=50)
plt.title('Distribuição de frequência da coluna Total Charges')
plt.axvline(services['Total Charges'].mean(), color='red', linestyle='dashed', linewidth=1)
plt.axvline(services['Total Charges'].median(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(services['Total Charges'].mode()[0], color='yellow', linestyle='dashed', linewidth=1)
plt.legend(['Média', 'Mediana', 'Moda'])
plt.xlabel('total_charges')
plt.ylabel('Frequência')

plt.subplot(2, 1, 2)
# Boxplot da coluna Total Charges horizontal
plt.boxplot(services['Total Charges'], vert=False)
# anotar a média, mediana e moda com plt.axvline e plt.legend
plt.axvline(services['Total Charges'].mean(), color='red', linestyle='dashed', linewidth=1)
plt.axvline(services['Total Charges'].median(), color='green', linestyle='dashed', linewidth=1)
plt.axvline(services['Total Charges'].mode()[0], color='yellow', linestyle='dashed', linewidth=1)
plt.legend(['Média', 'Mediana', 'Moda'])
plt.xlabel('total_charges')
plt.show()


### Variáveis categóricas

A Análise Univariada de variáveis categóricas pode ser feita usando um gráfico de barras. Para poder verificarmos a distribuição dos dados, podemos ordenar as barras por ordem crescente ou decrescente.

```python
services['Internet Service'].value_counts().plot(kind='bar')
```


In [ ]:
# Gráfico de barras para as colunas categóricas Internet Service, Offer
plt.figure(figsize=(10, 6))
plt.subplot(2, 2, 1)
# Plotar um gráfico de barras para a coluna Internet Service com plt.bar
plt.bar(services['Internet Service'].value_counts().index, services['Internet Service'].value_counts())
plt.title('Distribuição de frequência da coluna Internet Service')
plt.xlabel('internet_service')
plt.ylabel('Frequência')

# espaço entre os gráficos
plt.subplots_adjust(wspace=0.5)

plt.subplot(2, 2, 2)
# grafico de barras vertical para a coluna Offer
plt.barh(services['Offer'].value_counts().index, services['Offer'].value_counts(), color='orange')
plt.title('Distribuição de frequência da coluna Offer')
plt.xlabel('offer')
plt.ylabel('Frequência')
plt.show()

## Análise Bivariada
A análise bivariada consiste na análise entre duas variáveis. Ela pode ser feita para variáveis numéricas ou categóricas. Para variáveis numéricas, podemos visualizar a distribuição dos dados usando um gráfico de dispersão. Para variáveis categóricas, podemos visualizar a distribuição dos dados usando um gráfico de barras.

### Variáveis numéricas vs numéricas

#### Gráfico de dispersão (Scatter plot)
O Scatter Plot é um gráfico que mostra a relação entre duas variáveis numéricas, onde podemos começar a avaliar a correlação entre elas.


```python
services.plot.scatter(x='Monthly Charge', y='Total Charges')
```


In [ ]:
# scatterplot entre as colunas Monthly Charge e Total Charges
plt.figure(figsize=(10, 6))
plt.scatter(dataframe_telco['Tenure in Months'], dataframe_telco['Total Charges'])
plt.title('Dispersão entre as colunas Tenure in Months e Total Charges')
plt.xlabel('tenure_in_months')
plt.ylabel('total_charges')
plt.show()


##### Correlação e Coeficiente de Pearson
A Correlação é uma medida que indica a força e a direção da relação entre duas variáveis. O Coeficiente de Pearson é uma medida de correlação que varia de -1 a 1. Quando o coeficiente de Pearson é 1, as duas variáveis tem uma correlação positiva perfeita, ou seja, quando uma variável aumenta, a outra também aumenta. Quando o coeficiente de Pearson é -1, as duas variáveis tem uma correlação negativa perfeita, ou seja, quando uma variável aumenta, a outra diminui. Quando o coeficiente de Pearson é 0, as duas variáveis não tem correlação, ou seja, quando uma variável aumenta, a outra não aumenta nem diminui.



**Correlação de Pearson**


$$ r = \frac{\sum_{i=1}^{n} (x_i - \bar{x})(y_i - \bar{y})}{\sqrt{\sum_{i=1}^{n} (x_i - \bar{x})^2}\sqrt{\sum_{i=1}^{n} (y_i - \bar{y})^2}} $$

Onde: 
$$ \bar{x} = \frac{\sum_{i=1}^{n} x_i}{n} $$
$$ \bar{y} = \frac{\sum_{i=1}^{n} y_i}{n} $$
$$ n = \text{tamanho da amostra} $$
$$ x_i = \text{valor da variável x na posição i} $$
$$ y_i = \text{valor da variável y na posição i} $$
$$ r = \text{coeficiente de Pearson} $$
$$ \sum_{i=1}^{n} = \text{soma de i=1 até n} $$


#### Matriz de correlação
A matriz de correlação é uma tabela que mostra a correlação entre todas as variáveis numéricas. Ela é muito útil para vermos quais variáveis tem uma correlação maior com a variável que queremos prever.

```python
services.corr()
```

##### Heatmap
O Heatmap é um gráfico que mostra a matriz de correlação de uma forma mais visual. Ele mostra a correlação entre todas as variáveis numéricas usando cores. Quanto mais próximo de 1, mais forte é a correlação positiva. Quanto mais próximo de -1, mais forte é a correlação negativa. Quanto mais próximo de 0, mais fraca é a correlação.

```python
import seaborn as sns
sns.heatmap(services.corr(), annot=True)
```

In [ ]:
# scatterplot entre as colunas Tenure in Months e Total Charges com seaborn
plt.figure(figsize=(10, 6))
sns.regplot(x='Tenure in Months', y='Total Charges', data=dataframe_telco, line_kws={'color': 'red'})
plt.title('Dispersão entre as colunas Tenure in Months e Total Charges')
# correlação entre as colunas Tenure in Months e Total Charges na legenda
plt.legend([f'Correlação: {np.round(dataframe_telco["Tenure in Months"].corr(dataframe_telco["Total Charges"]), 2)}'])
plt.xlabel('tenure_in_months')
plt.ylabel('total_charges')
plt.show()


In [ ]:
# selecionar as colunas numéricas com services.select_dtypes
num_cols = services.drop(columns=["Count"]).select_dtypes(include=np.number).columns

services.corr(numeric_only=True)

In [ ]:

# heatmap da correlação entre as colunas do dataframe_telco com as correlações mais fortes
plt.figure(figsize=(10, 6))

# criar um heatmap com sns.heatmap
sns.heatmap(services[num_cols].corr(numeric_only=True), annot=True, cmap='Greens', fmt='.2f')

### Variáveis categóricas vs numéricas

Também podemos avaliar a relação entre duas variáveis, onde uma é categórica e a outra é numérica. Podemos usar o Boxplot para ver a média, mediana, moda, mínimo, máximo, etc. de uma variável numérica para cada valor da variável categórica.

```python
services.boxplot(column='Monthly Charge', by='Offer')
```

In [ ]:
# boxplot da coluna Total Charges para cada categoria da coluna Offer
plt.figure(figsize=(10, 6))
sns.boxplot(x='Offer', y='Total Charges', data=dataframe_telco)
plt.title('Boxplot da coluna Total Charges para cada categoria da coluna Offer')